In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

# from src.Basic_LLM_functions import LLM_connection
from src.Remove_accents import remove_accents,remove_special_characters,divide_string_into_words
from src.Spanish_words import spanish_words
from src.Basic_LLM_functions import LLM_connection

In [2]:
df=pd.read_csv("Databases/Parsed_htmls.csv")

In [2]:
LLM_connection.load_model_from_path("src/LLMs/Meta-Llama-3.1-8B-Instruct")

c:\Users\Rafa\miniconda3\envs\llama3_1_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.64s/it]


In [ ]:
LLM_connection.system_prompt="""
                                You will receive extensive details about a product. 
                                Please create a structured JSON object for the product, ensuring that keys are both clear and broadly applicable across similar products. 
                                Choose key names that are intuitive, concise, and standardized to enable easy understanding and reuse of this structure for other products.
                                Answer in spanish, both for the content and for the key names.
                                Only output the json object, do not include any other text.
                            """

In [64]:
import ast
def turn_info_to_json(text):
    for i in range(3):
        out=LLM_connection.Get_answer(text)
        try:
            output=ast.literal_eval(out[out.index("{"):].replace("`","").replace("`",""))
            return output
        except:
            pass
    return {}

In [ ]:
df.to_csv("Databases/Products_info_parsed_to_jsons.csv",index=False)

In [ ]:
Cleaned_general_info=[]
for i in tqdm(df.index):
    text=df.at[i,"General Information"]
    Cleaned_general_info.append(turn_info_to_json(text))
df["Cleaned_general_info"]=Cleaned_general_info

In [51]:
Cleaned_about_this_product=[]
for i in tqdm(df.index):
    text=df.at[i,"About This Product"]
    Cleaned_about_this_product.append(turn_info_to_json(text))
df["Cleaned_about_this_product"]=Cleaned_about_this_product

100%|██████████| 286/286 [1:08:26<00:00, 14.36s/it]


In [2]:
df=pd.read_csv("Databases/Products_info_parsed_to_jsons.csv")

In [3]:
import ast

all_info=[]
all_keys=set()
for i in tqdm(df.index):
    info={}
    info.update(ast.literal_eval(df.at[i,"Technical Details"]))
    info.update(ast.literal_eval(df.at[i,"Cleaned_general_info"]))
    info.update(ast.literal_eval(df.at[i,"Cleaned_about_this_product"]))

    all_info.append(info)
    all_keys.update(info.keys())

100%|██████████| 286/286 [00:00<00:00, 1888.57it/s]


In [4]:
all_keys={remove_accents(x).lower() for x in all_keys}
all_keys_list=list(all_keys)

In [5]:
#LLM_connection.system_prompt=f"""
#                            You will receive a Spanish string that may combine multiple words without spaces. Your task is to separate the words with spaces, preserving their original order and meaning.
#
#                            
#                            Do not rearrange the words.
#                            Do not change the gender or form of the words.
#                            Do not add extra words or connectors.
#                            Do not change verb tenses.
#                            Do not change the spelling of the words.
#
#                            The letters outputted must be the same as the input and in the same order, only separated by spaces.
#                            Example:
#
#                            Input: elamordemivida
#                            Output: el amor de mi vida
#                            Respond only with the separated words, without any additional text or explanations.
#                            """
#def divide_word_into_strings(word):
#    success=False
#    for i in range(3):
#        output=LLM_connection.Get_answer(word,max_new_tokens=50)
#        if output.replace(" ","").replace("_","").lower()==word.replace(" ","").replace("_","").lower():
#            success=True
#            break
#    
#    if success:
#        return output
#    print(f"Failed to divide word {word}, tried output: {output}")
#    return word
#LLM_connection.Get_answer("seincluyenlaspilas")

In [6]:
def NoTwoShortLetterWordsTogether(lista):
    for i in range(len(lista)-1):
        if len(lista[i])<=2 and len(lista[i+1])<=2:
            return False
    return True
keys_cleaned={}
for key in tqdm(all_keys_list):
    raw_key=key
    key=remove_special_characters(key)
    if len(key.split())>1:
        output_div=key.split()
    else:
        output_div=divide_string_into_words(spanish_words,key)
        if not(len(output_div)<=5 or NoTwoShortLetterWordsTogether(output_div)):
            output_div=[key]
    keys_cleaned[key]=" ".join(output_div)

100%|██████████| 544/544 [00:00<00:00, 890.80it/s]


In [7]:
new_keys=set(keys_cleaned.values())
new_keys={x.replace("unidad de","").replace("unidad","").replace("recomendado","").strip() for x in new_keys}

In [ ]:
final_keys={
    'accesorios'
    'Tiene ahorro energia',
    'alcance',
    'alimentacion',
    'almacenamiento',
    'amperaje',
    'apagado automatico',
    'apariencia',
    'aplicaciones',
    'autonomia',
    'bateria',
    'Tiempo de carga',
    'bolsa',
    'cabezal',
    'capacidad del deposito de polvo',
    'cepillo',
    'clase de reemision de polvo',
    'color',
    'componentes',
    'conectividad',
    'consumo',
    'detalles fisicos adicionales',
    'dimensiones del producto',
    'peso',
    'filtro',
    'funcion seca suelo',
    'garantia',
    'iluminacion',
    'inalambrico',
    'longitud cable',
    'marca',
    'material',
    'maxima presion de succion',
    'nivel de ruido',
    'modos de funcionamiento',
    'motor',
    'necesita baterias',
    'se incluyen las pilas',
    'tipos de superficie',
    'caracteristicas adicionales',
    }

In [36]:
LLM_connection.load_api(model="o1-preview",dot_env_path=".env2")

In [37]:
LLM_connection.system_prompt=f"""
                            You will be provided with a set of keys from dictionaries of information of similar products. 
                            Your task is to discern all the keys that refer to different things. To simplify the list of keys and avoid redundancy.
                            We only want to keep keys that may be relevant for data analysis and comparison. If 2 keys seem very similar, keep the more general one.
                            This means if a key is the plural of another one, just keep the singular term.

                            Return the result as a dict object in the following format:
                            {{"key1":{{"key1","key2", "key3", ...}}}}

                            Where the key1 is the key that you consider the most general and the values are the keys that you consider redundant.
                            Having this dictionary format allows us to know which keys are redundant and which key should be kept.

                            - Do not include any additional text, explanations, or formatting outside the set object.
                            - Remove characteristics that are too general, like: Special characteristics, additional features, etc, by putting them under: "Irrelevant characteristics"
                            Answer just with the dictionary of reduced keys
                            """

In [29]:
LLM_connection.api_model="gpt-4o"

In [38]:
out2=LLM_connection.Get_answer(f"Set of keys: {list(new_keys)}",max_new_tokens=10000)


NotFoundError: Error code: 404 - {'error': {'message': 'The model `o1-preview` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [20]:
out=LLM_connection.Get_answer(f"Set of keys: {list(new_keys)}",max_new_tokens=10000)

In [33]:
sum([len(x) for x in ast.literal_eval(out).values()])

228

In [35]:
ast.literal_eval(out)

{'nombre del producto': {'nom del modelo',
  'nombre',
  'nombre del modelo',
  'nombre modelo'},
 'tipo': {'tipo de bateria',
  'tipo de conectividad',
  'tipo de conexion',
  'tipo de controlador',
  'tipo de filtro',
  'tipo de material',
  'tipo de motor',
  'tipo de producto',
  'tipo de suelo',
  'tipo de uso',
  'tipos',
  'tipos de componentes',
  'tipos de filtro',
  'tipos de productos',
  'tipos de suelos',
  'tipos de superficie',
  'tipos des uci edad'},
 'peso': {'es deme did ade peso',
  'medida peso',
  'peso del producto',
  'peso es',
  'peso producto',
  'unida desde peso'},
 'dimensiones': {'dimensiones del paquete',
  'dimensiones del producto',
  'dimensionesypeso',
  'longitud',
  'longitud de la manguera',
  'longitud del cable',
  'longitud manguera',
  'tamano',
  'tamano pantalla'},
 'capacidad': {'capacidad bateria',
  'capacidad de contenido',
  'capacidad del deposito de polvo',
  'capacidad del recipiente',
  'capacidades'},
 'potencia': {'me did ade pote

In [67]:
all_words_in_new_keys=set()
for key in new_keys:
    all_words_in_new_keys.update(key.split())

In [69]:
new_keys

{'accesorios',
 'accesorios incluidos',
 'actualizaciones de software garantizadas hasta',
 'adv is os',
 'ahorro energia',
 'alcance',
 'alimentacion',
 'almacenamiento',
 'amp era je',
 'an no de lanzamiento',
 'apagado automatico',
 'apariencia',
 'aplicaciones',
 'asistentes',
 'atencion al cliente',
 'atributo unico',
 'autonomia',
 'avances',
 'avant ag es',
 'avant aj es',
 'aviso',
 'base',
 'base carga',
 'bateria',
 'bateria composicion',
 'bateria extra ib le',
 'baterias',
 'bateriaytiempodecarga',
 'beneficios',
 'bolsa',
 'cab ez al',
 'cabeza les',
 'cabeza lg ira tor io',
 'cable carga',
 'calidad y seguridad',
 'calificaciones',
 'camara',
 'cantidad',
 'cantidad de productos',
 'cantidad productos',
 'capacidad',
 'capacidad bateria',
 'capacidad de contenido',
 'capacidad del deposito de polvo',
 'capacidad del recipiente',
 'capacidades',
 'caracteristicas',
 'caracteristicas adicionales',
 'caracteristicas especiales',
 'caracteristicas generales',
 'caracteristica

In [119]:
#key_to_check=list(all_keys)[0]
#LLM_connection.system_prompt=f"""
#                            You will be provided with a set of keys from dictionaries of product information from vacuum cleaners, along with a specified key. 
#                            Your task is to identify all the keys in the set whose information in the dictionary would be the same as the specified key.
#
#                            Return the result strictly as a Python set in this format:
#                            {{"key1", "key2", "key3", ...}}.
#                            Dont put any additional text.
#                            """

In [33]:
LLM_connection.system_prompt=f"""
                            You will be provided with a set of keys from dictionaries of information of similar products. 
                            Your task is to discern all the keys that refer to different things. To simplify the list of keys and avoid redundancy.
                            We only want to keep keys that may be relevant for data analysis and comparison. If 2 keys seem very similar, keep the more general one.
                            This means if a key is the plural of another one, just keep the singular term.

                            Return the result as a set object in the following format:
                            {{"key1", "key2", "key3", ...}}

                            - Do not include any additional text, explanations, or formatting outside the set object.
                            - Remove characteristics that are too general, like: Special characteristics, additional features, etc.
                            Answer just with the set of reduced keys
                            """

In [53]:
all_keys_round=set(keys_cleaned.values()).copy()

previous_size=len(all_keys_round)
new_size=0
failed_rounds=0
while failed_rounds<3:
    previous_size=len(all_keys_round)
    out=LLM_connection.Get_answer(f"Set of keys {all_keys_round}",max_new_tokens=10000)
    try:
        all_keys_new_round=ast.literal_eval(out)
        if len(all_keys_new_round)<len(all_keys_round):
            all_keys_round=all_keys_new_round
    except:
        print("Error")
    print(len(all_keys_round),len(all_keys_new_round))
    new_size=len(all_keys_round)
    if new_size==previous_size:
        failed_rounds+=1
    else:
        failed_rounds=0

413 413
406 406
368 368
366 366
Error
366 366
Error
366 366
133 133
73 73
46 46
31 31
24 24
15 15
12 12
10 10
10 10
10 10
10 10


In [54]:
len(all_keys_round),len(set(keys_cleaned.values()))

(10, 452)

In [48]:
out=LLM_connection.Get_answer(f"Set of keys {set(keys_cleaned.values())}",max_new_tokens=10000)

In [55]:
all_keys_round

{'capacidad',
 'color',
 'consumo',
 'marca',
 'material',
 'nombre',
 'peso',
 'potencia',
 'tipo',
 'velocidad'}

In [50]:
len(ast.literal_eval(out))

168

In [52]:
all_keys_round

{'accesorios',
 'accesorios incluidos',
 'actualizaciones de software garantizadas hasta',
 'adv is os',
 'ahorro energia',
 'alcance',
 'alimentacion',
 'almacenamiento',
 'amp era je',
 'an no de lanzamiento',
 'apagado automatico',
 'apariencia',
 'aplicaciones',
 'asistentes',
 'atencion al cliente',
 'atributo unico',
 'autonomia',
 'avances',
 'avant ag es',
 'avant aj es',
 'aviso',
 'base',
 'base carga',
 'bateria',
 'bateria composicion',
 'bateria extra ib le',
 'baterias',
 'bateriaytiempodecarga',
 'beneficios',
 'bolsa',
 'cab ez al',
 'cabeza les',
 'cabeza lg ira tor io',
 'cable carga',
 'calidad y seguridad',
 'calificaciones',
 'camara',
 'cantidad',
 'cantidad de productos',
 'cantidad productos',
 'capacidad',
 'capacidad bateria',
 'capacidad de contenido',
 'capacidad del deposito de polvo',
 'capacidad del recipiente',
 'capacidades',
 'caracteristicas',
 'caracteristicas adicionales',
 'caracteristicas especiales',
 'caracteristicas generales',
 'caracteristica

In [ ]:
final_keys=all_keys_round.copy()

In [141]:
LLM_connection.system_prompt=f"""
                            You will be provided with 2 sets of keys pertaining to dictionaries of product information.
                            One set is a reduced version of the keys, and the other set is the original set of keys.
                            Your task is to group the keys from the original set into the reduced set.
                            You should group keys that refer to the same information together.
                            You should return a dictionary where the keys are the reduced keys and the values are sets of the original keys that belong to that group.
                            Notice that if a word is a plural version of another word, you should group them together.

                            Return the result as a Python dictionary in the following format:
                            {{"reduced_key1": {{"original_key1", "original_key2", "original_key3", ...}},...}}
                            Only return the dictionary object, do not include any additional text.
                            """
classified_keys=ast.literal_eval(LLM_connection.Get_answer(f"Set of reduced keys {final_keys}.\n Keys to group: {all_keys}"))


In [142]:
inverted_dict = {}
for main_key, values in classified_keys.items():
    for value in values:
        inverted_dict[remove_accents(value).lower()] = main_key

In [126]:
len(all_keys),len(set(inverted_dict.keys()))

(572, 304)

In [ ]:
##To do this we need the openai model, the open source models fall flat (even after trying much simpler stuff like only considering 10 elements at a time)
#LLM_connection.system_prompt=f"""
#                            You will be provided with a set of keys from dictionaries of information of similar products. 
#                            Your task is to identify all the keys that you would expect the information sheet to contain the same value, and group them together.
#
#                            Return the result as a dict object in the following format:
#                            {{"key1":{{"key1", "key2", "key3", ...}},"key5":{{"key6", "key7", "key8", ...}}}}
#
#                            - Do not include any additional text, explanations, or formatting outside the set object.
#                            - If a key is not related to any other key, it should be in a group by itself ("key":{{"key"}}).
#
#                            Example output dict: {{"location":{{"location","country","city"}}}} (These all refer to the location)
#                            """

In [ ]:
#To do this we need the openai model, the open source models fall flat (even after trying much simpler stuff like only considering 10 elements at a time)
LLM_connection.system_prompt=f"""
                            You will be provided with a set of keys from dictionaries of information of similar products. 
                            Your task is to identify all the keys that you would expect the information sheet to contain the same value.

                            Return the result as a set object in the following format:
                            {{"key1", "key2", "key3", ...}}

                            - Include the given key in the set if it exists in the original set.
                            - Do not include any additional text, explanations, or formatting outside the set object.

                            Example: key:"location" {{"location","country","city"}} (These all refer to the location)
                            """

In [66]:
#out=LLM_connection.Get_answer(f"Here is the set of all keys: {all_keys}\n Key to check: {key_to_check}")

In [85]:
key_sets = {}  # Dictionary to store the result
all_keys_modified = set(all_keys)  # Start with the full set of keys

# Initialize tqdm progress bar with the total number of keys
with tqdm(total=len(all_keys), desc="Processing keys", unit="key") as pbar:
    while all_keys_modified:
        key_to_check = next(iter(all_keys_modified))  # Take one key to check
        # Call the LLM for this key
        output_set = turn_info_to_json(f"Here is the set of all keys: {all_keys_modified}\n Key to check: {key_to_check}")
        
        if not output_set:  # Check if output_set is empty
            output_set = set([key_to_check])
        
        output_set.add(key_to_check)
        # Parse and add the result to the dictionary
        key_sets[key_to_check] = output_set
        
        true_output_set = set(x for x in output_set if x in all_keys_modified)
        # Remove keys from all_keys_modified that are part of the output_set
        all_keys_modified -= output_set
        
        # Update the progress bar
        pbar.update(len(true_output_set))

Processing keys: 100%|██████████| 572/572 [00:48<00:00, 11.75key/s]


In [ ]:
len(inverted_dict.keys()),len(all_keys)

(146, 572)

In [43]:
out

'{"unidadDeMedidaDePotencia":{"unidad_de_medida_de_potencia","unidadDeMedidaDePotencia","unidad_de_medida_potencia","unidad_medida_potencia","unidadDeMedidaDePotencia"},"numeroProductos":{"numero_productos","Numero de productos","numeroDeProductos","Numero de productos","cantidad_productos"},"tiempoDeCarga":{"tiempo_carga","tiempoDeCarga","tiempo de carga","tiempo_de_carga","unidadDeTiempoDeCarga"},"peso":{"peso_producto","peso del producto","pesoDelProducto","peso","peso_unidades"},"tipoDeFiltro":{"tipoFiltro","tipo_de_filtro","tipoDeFiltro","tipos_de_filtro","tipo de filtro"},"funcionalidades":{"funcionalidades","funcionalidades adicionales","funcionalidades_especiales","funcionalidadesEspeciales","funcionalidades especiales"},"recomendacion_superficie":{"recomendacionSuperficie","recomendacion de superficie","recomendacion_de_superficie","recomendacionesDeSuperficie","superficie recomendada"},"nivel_ruido":{"nivel_de_ruido","nivel_silencio","niveles_de_ruido","nivelRuido","niveles_d

In [52]:
df.to_csv("Databases/Products_info_parsed_to_jsons.csv",index=False)